In [36]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from typing import List
import pandas as pd
import sacrebleu
from rouge_score import rouge_scorer
import evaluate
from rouge_score import rouge_scorer


rouge = evaluate.load('rouge')
sacrebleu = evaluate.load("sacrebleu")
chrf = evaluate.load("chrf")


from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("iocuydi/llama-2-amharic-3784m")

tokenizer_config.json: 100%|██████████| 745/745 [00:00<00:00, 317kB/s]
tokenizer.model: 100%|██████████| 899k/899k [00:00<00:00, 6.62MB/s]
tokenizer.json: 100%|██████████| 1.84M/1.84M [00:00<00:00, 4.17MB/s]
special_tokens_map.json: 100%|██████████| 411/411 [00:00<00:00, 654kB/s]


In [37]:
# d = pd.read_csv('./amharic_title_generation/gpt4_responses-amharic_title_generation.csv').input.values.tolist()


In [50]:

data = pd.read_csv('./amharic_title_generation/gpt4_responses-amharic_title_generation.csv')
# print(data.Datasource.value_counts())
data.tail()

,instruction,input,ouptput,datasource,response
1295,የሚቀጥለው ዜና አርእስት ፃፍ,መጋቢት የምስሉ መግለጫ ጠቅላይ ሚኒስትር ቴሬዛ ሜይ የብሔራዊ ደህንነት ም...,ጠ ሚ ሜይ ሞስኮ በሰላዩ መመረዝ ጉዳይ አስተያየት መስጠት አለባት ብለዋል,amharic_title_generation,ትዮጵያ ከኮሮናቫይረስ የሚሞቱት ሰዎች ቁጥር ወደ 385 ወጥቶ
1296,የሚቀጥለው ዜና አርእስት ፃፍ,ነሐሴ ይህ ፕሮጀክት የተጀመረው እንደ ቀልድ ነበር። ጓደኛሞች በጋራ ሰብሰ...,አፍሪካዊው ወጣት ለወባ በሽታ መፍትሄ ይዞ መጥቷል,amharic_title_generation,ይን ነይ ሁኔታ ተመለሰ። ይህንን ለማፈራረስ የታዘዙ ምርመ
1297,የሚቀጥለው ዜና አርእስት ፃፍ,ወደድንም ጠላንም ዶናልድ ትራምፕ ስልጣን ላይ ከወጡ አንድ አመት ሞልቷቸዋ...,ወደድንም ጠላንም ዶናልድ ትራምፕ ስልጣን ላይ ከወጡ አንድ አመት ሞልቷቸዋል።,amharic_title_generation,ዶናልድ ትራምፕ የሚኒስቴር በጠቅላይ ሚኒስቴር ማእከል ተሰርዮ ከሆኑት ስሙ...
1298,የሚቀጥለው ዜና አርእስት ፃፍ,ጥቅምት የፎቶው ባለመብት የትራምፕ አስተዳደር በሜክሲኮ እና በአሜሪካ መካ...,ትራምፕ ስደተኞችን ለማባረር አዲስ ዕቅድ እያቀዱ ነው,amharic_title_generation,በሚቀጥለው ዜና አርእስት ፃፍ:\n\nአሜሪካ ፕሬዚደንት ዶናልድ ትራምፕ በ...
1299,የሚቀጥለው ዜና አርእስት ፃፍ,መስከረም የፎቶው ባለመብት እብዱ የአሜሪካ ፕሬዝዳንት ትራምፕ ለሰሜን ኮሪ...,ኪም ጆንግ ኡን ፡ እብዱ ትራምፕ የኑክሊዬር መርሃ ግብርን አስፈላጊነት አ...,amharic_title_generation,መትከል ይጠበቃል። በአገሪቱ መቀየሪያዎች የሚዘረጋው በ


In [53]:
data=data[data.datasource=="amharic_title_generation"]
# data.drop_duplicates(subset='Input Text', keep="last")
# data = data.head(1300)
data = data.fillna('')
data.tail()

,instruction,input,ouptput,datasource,response
1295,የሚቀጥለው ዜና አርእስት ፃፍ,መጋቢት የምስሉ መግለጫ ጠቅላይ ሚኒስትር ቴሬዛ ሜይ የብሔራዊ ደህንነት ም...,ጠ ሚ ሜይ ሞስኮ በሰላዩ መመረዝ ጉዳይ አስተያየት መስጠት አለባት ብለዋል,amharic_title_generation,ትዮጵያ ከኮሮናቫይረስ የሚሞቱት ሰዎች ቁጥር ወደ 385 ወጥቶ
1296,የሚቀጥለው ዜና አርእስት ፃፍ,ነሐሴ ይህ ፕሮጀክት የተጀመረው እንደ ቀልድ ነበር። ጓደኛሞች በጋራ ሰብሰ...,አፍሪካዊው ወጣት ለወባ በሽታ መፍትሄ ይዞ መጥቷል,amharic_title_generation,ይን ነይ ሁኔታ ተመለሰ። ይህንን ለማፈራረስ የታዘዙ ምርመ
1297,የሚቀጥለው ዜና አርእስት ፃፍ,ወደድንም ጠላንም ዶናልድ ትራምፕ ስልጣን ላይ ከወጡ አንድ አመት ሞልቷቸዋ...,ወደድንም ጠላንም ዶናልድ ትራምፕ ስልጣን ላይ ከወጡ አንድ አመት ሞልቷቸዋል።,amharic_title_generation,ዶናልድ ትራምፕ የሚኒስቴር በጠቅላይ ሚኒስቴር ማእከል ተሰርዮ ከሆኑት ስሙ...
1298,የሚቀጥለው ዜና አርእስት ፃፍ,ጥቅምት የፎቶው ባለመብት የትራምፕ አስተዳደር በሜክሲኮ እና በአሜሪካ መካ...,ትራምፕ ስደተኞችን ለማባረር አዲስ ዕቅድ እያቀዱ ነው,amharic_title_generation,በሚቀጥለው ዜና አርእስት ፃፍ:\n\nአሜሪካ ፕሬዚደንት ዶናልድ ትራምፕ በ...
1299,የሚቀጥለው ዜና አርእስት ፃፍ,መስከረም የፎቶው ባለመብት እብዱ የአሜሪካ ፕሬዝዳንት ትራምፕ ለሰሜን ኮሪ...,ኪም ጆንግ ኡን ፡ እብዱ ትራምፕ የኑክሊዬር መርሃ ግብርን አስፈላጊነት አ...,amharic_title_generation,መትከል ይጠበቃል። በአገሪቱ መቀየሪያዎች የሚዘረጋው በ


In [40]:
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed
[nltk_data]     (_ssl.c:1131)>


False

In [43]:
def calculate_bleu_score(references: List[List[str]], candidates: List[str]) -> float:
    scores = []
    for ref, cand in zip(references, candidates):
        candidate_tokens = nltk.word_tokenize(cand)
        reference_tokens = [nltk.word_tokenize(r) for r in ref]
        
        score = sentence_bleu(reference_tokens, candidate_tokens)
        scores.append(score)

    
    print(f" sentence bleu : {sum(scores) / len(scores) if scores else 0}")

    bleu = sacrebleu.compute(predictions=candidates, references=references)
    print(f"SacreBLEU score: {bleu}")

    chr = chrf.compute(predictions=candidates, references=references, word_order=2)
    print(f"corpus_chrf score: {chr}")

    results = rouge.compute(predictions=candidates, references=references)
    print(f"Rogue results: {results}")

    # scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    # scores = scorer.score(candidates,references)
    # print(score)

   





In [44]:
calculate_bleu_score(data.gold_label.values.tolist(), data.response.values.tolist())


 sentence bleu : 4.327470820988156e-233
SacreBLEU score: {'score': 0.03064286031726817, 'counts': [577, 37, 10, 5], 'totals': [112427, 107476, 102549, 97705], 'precisions': [0.5132219128856947, 0.034426290520674384, 0.009751435898936118, 0.005117445371270662], 'bp': 1.0, 'sys_len': 112427, 'ref_len': 40130}
corpus_chrf score: {'score': 5.636147328468981, 'char_order': 6, 'word_order': 2, 'beta': 2}
Rogue results: {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}
